In [2]:
# Import required modules
from selenium import webdriver
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
# Initiate driver and open the link
driver = webdriver.Chrome("/home/faysal/Documents/utilities/chromedriver")
driver.get("https://internationalopenacademy.com/collections/en")
time.sleep(4)


# Select a safer range to load more pages
for _ in range(20):
    try:
        # Close the pop-up
        driver.find_element_by_css_selector("[title='Close']").click()
    except:
        pass

    time.sleep(3)
    
    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Close the pop-up if any
    try:
        driver.find_element_by_css_selector("[title='Close']").click()
    except:
        pass
    
    # Click on load more button
    try:
        nextPage = driver.find_element_by_class_name("btn.btn--large.btn--load-more")
        nextPage.click()
        time.sleep(6)
    except:
        print("No more page to load!")
        break

No more page to load!


In [4]:
# To store course links
courseLink = []

for cont in driver.find_elements_by_class_name("grid-view-item"):
    try:
        # Scrape course links
        courseLink.append(cont.find_element_by_css_selector(".product-grid--title a").get_attribute("href"))
    except:
        courseLink.append("na")
driver.close()

In [5]:
# This function scrapes course info from individual course links
def scrapeCourseInfo(url):
    """url = course link,
    return = course df"""
    
    # Make request
    r = requests.get(url)
    s = BeautifulSoup(r.text, "lxml")
    
    # Scrape course title
    try:
        courseTitle = s.find(class_="product-details-product-title").text.strip()
    except:
        courseTitle = "na"
    
    # Scrape meta description
    try:
        metaDes = s.find("p").text.strip()
    except:
        metaDes = "na"
    
    # Scrape offer price
    try:
        offerPrice = s.find(id="ProductPrice").text.strip().replace("€", "")
    except:
        offerPrice = None
    
    # Create a df off scraped variables
    df = pd.DataFrame({
        "courseTitle":courseTitle,
        "courseLink":url,
        "metaDescription":metaDes,
        "offerPrice":offerPrice
    }, index=[0])
    
    return df

In [6]:
%%time
# Scrape course info
finalDf = pd.concat(list(map(scrapeCourseInfo, courseLink))).reset_index(drop=True)

CPU times: user 3.46 s, sys: 86.6 ms, total: 3.54 s
Wall time: 2min 4s


In [6]:
# Preview the data
finalDf.head(10)

,courseTitle,courseLink,metaDescription,offerPrice
0,120-Hour TESOL Certificate,https://internationalopenacademy.com/collectio...,Teaching English gives you the freedom to work...,119.00
1,120-Hour TEFL Certificate,https://internationalopenacademy.com/collectio...,Teaching English gives you the freedom to work...,119.00
2,American Sign Language,https://internationalopenacademy.com/collectio...,ASL is the third most-used language in the Uni...,119.00
3,Master Herbalist,https://internationalopenacademy.com/collectio...,Herbalism and incorporating herbs into your ev...,99.00
4,Criminal Psychology,https://internationalopenacademy.com/collectio...,Take a deep dive into the world of Criminal Ps...,99.00
5,Bartender & Barista Training,https://internationalopenacademy.com/collectio...,"Bartending is a globally required, in demand s...",119.00
6,Car Detailing Fundamentals,https://internationalopenacademy.com/collectio...,Keep your car looking brand new without spendi...,249.00
7,Excel Training,https://internationalopenacademy.com/collectio...,Microsoft Excel is used in almost every office...,99.00
8,Criminology and Profiling,https://internationalopenacademy.com/collectio...,Profiling has become a subject of significant ...,119.00
9,Drawing and Illustration for Beginners,https://internationalopenacademy.com/collectio...,Created with popular YouTube artist Shayda Cam...,99.00
